In [ ]:
import os
os.chdir("/opt/project")

import json
import psutil
import numpy as np

import keras
keras.mixed_precision.set_global_policy("mixed_float16")

from figures import ModelFigureGenerator
from datasets import CardiogramSequence, get_files
from model import ModelSpesification, get_model
from helpers import ResourcePath, CardiogramLead

In [ ]:
model_id = 0
model_spec = ModelSpesification(model_id, tags=("best",))

In [ ]:
# Model settings
lr = 1e-3
batch_size = 64
val_split = 0.02
shuffle_on_epoch_end = True
worker_num = psutil.cpu_count(logical=True) - 2

opt = keras.optimizers.Adam(lr)
loss = keras.losses.BinaryCrossentropy()

callbacks = [
    # Learning Optimizers
    keras.callbacks.EarlyStopping(patience=9, min_delta=1e-5),
    keras.callbacks.ReduceLROnPlateau(patience=7, min_lr=lr * 1e-2),
    # Logs
    keras.callbacks.TensorBoard(log_dir=model_spec.log_dir, write_graph=False),
    keras.callbacks.CSVLogger(model_spec.log_dir / "training.log", append=False),
    # Checkpoints
    keras.callbacks.ModelCheckpoint(
        model_spec["best"].model_dir / "model.keras", save_best_only=True
    ),
]

channels = [CardiogramLead.V3, CardiogramLead.V4, CardiogramLead.V5]
train_seq, valid_seq = CardiogramSequence.get_train_and_val(
    get_files("input"), get_files("label"), batch_size=batch_size,
    val_split=val_split, leads=channels, shuffle=shuffle_on_epoch_end,
    workers=worker_num, use_multiprocessing=True, max_queue_size=20
)

# If you are continuing an interrupted section, uncomment the line below:
# model = keras.models.load_model(model_spec.model_dir / "model.keras", compile=False)
model = get_model(n_classes=train_seq.n_classes,
                  n_channels=len(channels), batch_size=batch_size)
model.compile(loss=loss, optimizer=opt)

with open(model_spec.model_dir / "struct.json", "w") as file:
    json.dump(keras.saving.serialize_keras_object(model), file, indent=4)

In [ ]:
# Train neural network
history = model.fit(train_seq,
                    epochs=70,
                    initial_epoch=0,
                    callbacks=callbacks,
                    validation_data=valid_seq,
                    verbose=1)

# Save final result
model.save(model_spec.model_dir / "model.keras")

In [ ]:
# Evaluate the model on the test set
seq = CardiogramSequence([ResourcePath.TEST_INPUTS / "ecg_tracings.hdf5"], None,
                  "tracings", batch_size=1, drop_last=False, leads=channels)
y_score = model_spec.model.predict(seq, verbose=1)

np.save(model_spec.model_dir / "prediction.npy", y_score)

In [ ]:
# Generate figures and tables
stats = ModelFigureGenerator(model_spec)

stats.generate_table_two()
stats.generate_supplementary_table_one()
stats.generate_supplementary_figure_one()

In [ ]:
# Inspect training logs
%load_ext tensorboard
%tensorboard --logdir {model_spec.log_dir} --bind_all